In [1]:
# import pandas as pd

In [2]:
# data=pd.read_csv('processed_data_all.csv')
# data.head(10)

In [3]:
# len(data)

In [4]:
from concurrent.futures import ThreadPoolExecutor
from transformers import pipeline

# Load model analisis sentimen bahasa Indonesia dengan tiga kelas sentimen
# sentiment_analysis = pipeline("text-classification", model="ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa")


2024-04-27 22:22:30.189452: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-27 22:22:31.472539: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
# sentimen_pipe = pipeline("summarization", model="cahya/bert2gpt-indonesian-summarization")

In [6]:
# teks = "Kumaha barudak? Well!Yuk ramaikan Pesta Anak Nongkrong yang diselenggarakan PAN x JakCloth! 10-12 November 2023 di Kiara Park Bandung.Artis-artis Bluesquad akan hadir di tanggal 11 ya! Ada Tipe-X, J-Rocks, Of Triad dan banyak lainnya. Nikmati juga berbagai jajanan dan produk fashion UMKM lokal lainnya!Tong hilap, nya!"

# # Menampilkan hasil analisis sentimen
# hasil_sentimen = sentimen_pipe(teks)
# print(hasil_sentimen[0])

In [7]:
sentimen_pipe1 = pipeline("text-classification", model="taufiqdp/indonesian-sentiment")

In [8]:
teks = "Jalan khusus batu bara di Jambi kini sudah rampung sejauh 80 persen. Jalan ini menjadi solusi dari kejadian sebelumnya yaitu macet horor selama 22 jam di Jambi."

# Menampilkan hasil analisis sentimen
hasil_sentimen = sentimen_pipe1(teks)
print(hasil_sentimen)

[{'label': 'negatif', 'score': 0.9514365792274475}]


In [9]:
# sentimen_pipe2 = pipeline("text-classification", model="crypter70/IndoBERT-Sentiment-Analysis")

In [10]:

# # Menampilkan hasil analisis sentimen
# hasil_sentimen = sentimen_pipe2(teks)
# print(hasil_sentimen)

In [11]:
# sentimen_pipe3 = pipeline("sentiment-analysis", model="indolem/indobert-base-uncased")

In [12]:
# Menampilkan hasil analisis sentimen
hasil_sentimen = sentimen_pipe3(teks)
print(hasil_sentimen)

NameError: name 'sentimen_pipe3' is not defined

In [ ]:
# sentimen_pipe = pipeline("sentiment-analysis", model="indobenchmark/indobert-large-p2")

In [ ]:
# teks = "Kumaha barudak? Well!Yuk ramaikan Pesta Anak Nongkrong yang diselenggarakan PAN x JakCloth! 10-12 November 2023 di Kiara Park Bandung.Artis-artis Bluesquad akan hadir di tanggal 11 ya! Ada Tipe-X, J-Rocks, Of Triad dan banyak lainnya. Nikmati juga berbagai jajanan dan produk fashion UMKM lokal lainnya!Tong hilap, nya!"

# # Menampilkan hasil analisis sentimen
# hasil_sentimen = sentimen_pipe(teks)
# print(hasil_sentimen)

In [ ]:
# # Fungsi untuk melabeli sentimen pada setiap baris data
# def label_sentiment(text):
#     # Analisis sentimen menggunakan model Transformer
#     result = sentiment_analysis(text)
#     # Ambil label sentimen teratas
#     top_sentiment = result[0]
#     label = top_sentiment['label']
#     score = top_sentiment['score']
#     print(label)
#     return label, score

# # Fungsi untuk melabeli sentimen menggunakan multi-threading
# def label_sentiment_multithread(text_list):
#     with ThreadPoolExecutor() as executor:
#         results = executor.map(label_sentiment, text_list)
#     return list(results)

# # Bagi data menjadi batch agar dapat diproses menggunakan multi-threading
# batch_size = 1000
# num_batches = len(data) // batch_size + 1
# batches = [data[i*batch_size:(i+1)*batch_size] for i in range(num_batches)]

# # Melabeli sentimen untuk setiap batch menggunakan multi-threading
# labeled_data = []
# for batch in batches:
#     texts = batch['content'].tolist()
#     labels = label_sentiment_multithread(texts)
#     labeled_data.extend(labels)

# # Memasukkan hasil label sentimen ke dalam DataFrame
# data['sentiment_label'], data['sentiment_score'] = zip(*labeled_data)

In [ ]:
# Simpan data yang telah dilabeli ke dalam file CSV
# data.to_csv('label_all.csv', index=False)